In [ ]:
# # 如果你不确定当前环境里还有什么其他冲突包，最保险的方法是新建一个环境。
# # 创建一个名为 'torch_env' 的新环境，指定 Python 3.11（更稳定）
# conda create -n torch_env python=3.11

# # 激活新环境
# conda activate torch_env

# # 安装 PyTorch (替换为你的 CUDA 版本)
# conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
# %pip 会自动定位到当前 Jupyter Kernel 的 Python 解释器，确保包被安装到正确的位置。而 !pip 调用的是系统 Shell 中的 pip，可能安装到完全不同的 Python 环境中，导致你“装了但还是找不到”的情况。

In [ ]:
# # 激活你的环境（如 pytorch_GPU_cuda）
# !conda activate pytorch_GPU_cuda # ! call system shell to activate environment  
# # 卸载
# !conda uninstall torchvision
# # 重新安装（与 PyTorch 版本匹配）
# !conda install torchvision -c pytorch

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# 检查是否有 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

使用设备: cuda


In [3]:
#重装后，在 Python 中运行： sometine restart   the kernel will helpful  
from PIL import Image
print(Image.__version__)

11.3.0


In [5]:
import torchvision
print(torchvision.__version__)
print(torchvision.__file__)  # 查看安装位置

0.20.1
c:\Users\zhen-\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\__init__.py


In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [7]:
# PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [13]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
# Creating Models
# To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the accelerator such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

#device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [11]:
#We also check the model’s performance against the test dataset to ensure it is learning.

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306524  [   64/60000]
loss: 2.289540  [ 6464/60000]
loss: 2.274071  [12864/60000]
loss: 2.270713  [19264/60000]
loss: 2.261742  [25664/60000]
loss: 2.221151  [32064/60000]
loss: 2.234284  [38464/60000]
loss: 2.197087  [44864/60000]
loss: 2.199460  [51264/60000]
loss: 2.167277  [57664/60000]
Test Error: 
 Accuracy: 46.6%, Avg loss: 2.160701 

Epoch 2
-------------------------------
loss: 2.169291  [   64/60000]
loss: 2.154606  [ 6464/60000]
loss: 2.102417  [12864/60000]
loss: 2.122481  [19264/60000]
loss: 2.074846  [25664/60000]
loss: 2.007193  [32064/60000]
loss: 2.038663  [38464/60000]
loss: 1.959086  [44864/60000]
loss: 1.970608  [51264/60000]
loss: 1.898413  [57664/60000]
Test Error: 
 Accuracy: 56.0%, Avg loss: 1.895288 

Epoch 3
-------------------------------
loss: 1.925846  [   64/60000]
loss: 1.892266  [ 6464/60000]
loss: 1.787420  [12864/60000]
loss: 1.830475  [19264/60000]
loss: 1.717199  [25664/60000]
loss: 1.666831  [32064/600